# Systems Immunogenetics Project

## Expression Array DE and Pathway Analysis Workflow

### McWeeney Lab, Oregon Health & Science University

** Authors: Gabrielle Choonoo (choonoo@ohsu.edu) and Michael Mooney (mooneymi@ohsu.edu) **

## Introduction

This is the step-by-step workflow for the DE and pathway analysis of pre-processed expression from the Bat Array data, including plots for DE genes and GO pathways.

Required Files:
* Raw expression data. This can the `raw.exprs` object created in the array QA/QC workflow (SIG_Array_QA_QC_Workflow.ipynb): [[Download here]](https://raw.githubusercontent.com/mooneymi/systems_immunogenetics/master/SIG_Array_QA_QC_Workflow.ipynb)
* This notebook (notebook.ipynb): [[Download here]](https://raw.githubusercontent.com/gchoonoo/DE_array_analysis/master/notebook.ipynb)
* The R script `functions.R`: [[Download here]](https://raw.githubusercontent.com/gchoonoo/DE_array_analysis/master/functions.R)

Required R packages:
- `gdata`: 
- `plyr`: 
- `oligo`:
- `pd.mogene.2.1.st`:
- `mogene21sttranscriptcluster.db`:
- `limma`: [https://bioconductor.org/packages/release/bioc/html/limma.html](https://bioconductor.org/packages/release/bioc/html/limma.html)
- `Heatplus`:
- `reshape2`:
- `ReportingTools`:
- `hwriter`:
- `ggplot2`:
- `clusterProfiler`:
- `XML`:
- `GOstats`:
- `genefilter`:

**Note: this notebook can also be downloaded as an R script (only the code blocks seen below will be included): [[Download R script here]](https://raw.githubusercontent.com/gchoonoo/DE_array_analysis/master/notebook.r)

** All code is available on GitHub: [https://github.com/gchoonoo/DE_array_analysis](https://github.com/gchoonoo/DE_array_analysis) **



# Step 1. Load Necessary R Functions and Libraries

In [19]:
## Load libraries and functions for DE and pathway analysis
source('functions.R')

# Step 2: Load Raw Expression Data

In [6]:
## Set data directory and load raw expression data
## You will have to change the file name and paths
data_dir = '/Users/mooneymi/Documents/MyDocuments/SystemsImmunogenetics/Expression/Bat_Virus_Array/data'
raw_exprs_file = 'bat_virus_raw_exprs_2-FEB-2016.rda'
## This will load the raw.exprs object 
load(file.path(data_dir, raw_exprs_file))

# Step 3: Perform DE Analysis

## Step 3a: Select Samples and Normalize Expression Data

In [7]:
## In this case we are selecting samples based on extreme values of the day 4 weight percentage
## We will create a 'Category' variable to indicate the two sample groups to compare
pData(raw.exprs)$Category = NA
pData(raw.exprs)[which(pData(raw.exprs)[,'D4_percent'] < 0.85),'Category'] <- 'Sensitive'
pData(raw.exprs)[which(pData(raw.exprs)[,'D4_percent'] > 0.98),'Category'] <- 'Resistant'
## Subset the ExpressionSet
raw.exprs.filter <- raw.exprs[, !is.na(raw.exprs$Category)]

In [8]:
dim(raw.exprs)

Features  Samples 
 1416100       96

In [9]:
dim(raw.exprs.filter)

Features  Samples 
 1416100       40

In [16]:
## Normalize the data
norm.exprs.filter = rma(raw.exprs.filter, normalize=TRUE, target="core")

Background correcting
Normalizing
Calculating Expression


In [12]:
# Save normalized expression as .csv file (optional)
output_normalized_expression(norm.exprs=norm.exprs.filter, save.dir=data_dir)

Saving normalized expression to file: 
/Users/mooneymi/Documents/MyDocuments/SystemsImmunogenetics/Expression/Bat_Virus_Array/data/normalized_expression.csv


## Step 3b: Filter the Features

In [17]:
## This function removes transcript cluster IDs that do not map to an Entrez gene
## Also, if multiple transcript cluster IDs map to a gene, the one with the highest IQR is kept
norm.exprs.filter = filter_features(norm.exprs=norm.exprs.filter, save.dir=getwd())

Saving probe mapping to file: 
/Users/mooneymi/Documents/MyDocuments/SystemsImmunogenetics/Expression/Bat_Virus_Array/de_analysis/probe_mapping.txt


In [18]:
dim(norm.exprs.filter)

Features  Samples 
   24555       40

## Step 3c: Perform DE Analysis and Save Results

In [ ]:
# Compute DE analysis and save table of results
de_table = de_analysis_table(norm.exprs=norm.exprs.filter, category='Category', probe_mapping_file='probe_mapping.txt')

In [ ]:
head(de_table)

# Step 4: Perform Pathway Analysis

In [ ]:
# GO stats enrichment analysis of DE genes
path_results = pathway_analysis(norm.exprs=norm.exprs.filter, de_table=de_table, pvalue_cutoff=0.05)

In [ ]:
head(path_results)

## Step 4a: Pathway Visualization

In [ ]:
# Set parameter equal to either Odds Ratio (OddsRatio) or Gene Ratio (generatio)
pathway_plots(path_results=path_results, N=10, parameter='OddsRatio', pvalue='adjusted')

#### Last Updated: 10-Feb-2016